### 참고
- 넷플릭스
    - 슬로건: 모든 것이 추천
    - https://medium.com/netflixtechblog/55838468f429 : 넷플릭스 추천 알고리즘 구현에 대한 기술 블로그 공개

## 1. 연구 목표
- 사용자 평점 데이터를 기반으로 사용자를 특정, 예측해 추천 시스템 구축
- 회귀 처리, 회귀 평가, 추천시스템에 대한 이해
- FastFM(third party 알고리즘 사용: 인수분해머신 기능 지원)
  - 윈도우에서는 컴파일 후 설치 불가/ 리눅스에서 설치해 진행

## 2. 데이터 수집/확보
- ml-100k.zip 파일
- 영화 정보 데이터(고객 정보/영화 정보/평점 정보)

In [1]:
import pandas as pd

In [5]:
users = pd.read_csv('../table/ml-100k/u.user', sep='|', header=None)

In [7]:
users.columns = ['uid','age','m','job','zip_code']
users.tail(2)

,uid,age,m,job,zip_code
941,942,48,F,librarian,78209
942,943,22,M,student,77841


In [16]:
cols = ['mid','title','release_date','video_release_date','imdb_url']
movies = pd.read_csv('../table/ml-100k/u.item', sep='|', header=None, encoding='latin1', names=cols, usecols=range(5))
movies.tail(3)

,mid,title,release_date,video_release_date,imdb_url
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998)
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...
1681,1682,Scream of Stone (Schrei aus Stein) (1991),08-Mar-1996,NaN,http://us.imdb.com/M/title-exact?Schrei%20aus%...


In [15]:
movies.shape

(1682, 5)

In [20]:
ratings = pd.read_csv('../table/ml-100k/u.data', sep='\t', header=None, names=['uid','mid','rating','unix_timestamp'])
ratings.head(2)

,uid,mid,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742


In [21]:
# unix_timestamp는 1970년 1월 1일 00시 00분 00초부터 현재까지 경과된 시간 +09:00
# 시간 형식 변경
ratings['date'] = pd.to_datetime( ratings.unix_timestamp, unit='s' )
ratings.head(2)

,uid,mid,rating,unix_timestamp,date
0,196,242,3,881250949,1997-12-04 15:55:49
1,186,302,3,891717742,1998-04-04 19:22:22


In [ ]:
# 3. 데이터 준비/품질향상/전처리

In [ ]:
# 4. 데이터 분석(통계적,시각적)

In [ ]:
# 5. 예측모델 구축(머신러닝기반)

In [ ]:
# 6. 시스템 통합(서비스에 반영 ex. OTT, 쇼핑몰)